In [15]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [10]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
t1 = time.time()

print(f'Send took {(t1 - t0):.2f} seconds')

producer.flush()

t2 = time.time()
print(f'Flush took {(t2 - t1):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
Send took 0.51 seconds
Flush took 0.00 seconds


In [12]:
import pandas as pd
data = "green_tripdata_2019-10.csv.gz"

cols = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

df_green = pd.read_csv(data, usecols=cols)

In [13]:
df_green

lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  DOLocationID  \
0       2019-10-01 00:26:02   2019-10-01 00:39:58           112           196   
1       2019-10-01 00:18:11   2019-10-01 00:22:38            43           263   
2       2019-10-01 00:09:31   2019-10-01 00:24:47           255           228   
3       2019-10-01 00:37:40   2019-10-01 00:41:49           181           181   
4       2019-10-01 00:08:13   2019-10-01 00:17:56            97           188   
...                     ...                   ...           ...           ...   
476381  2019-10-31 23:30:00   2019-11-01 00:00:00            65           102   
476382  2019-10-31 23:03:00   2019-10-31 23:24:00           129           136   
476383  2019-10-31 23:02:00   2019-10-31 23:23:00            61           222   
476384  2019-10-31 23:42:00   2019-10-31 23:56:00            76            39   
476385  2019-10-31 23:23:00   2019-10-31 23:56:00            56           215   

        passenger_count  trip_distance  tip_amount  
0                   1.0           5.88        0.00  
1                   1.0           0.80        0.00  
2                   2.0           7.50        0.00  
3                   1.0           0.90        0.00  
4                   1.0           2.52        2.26  
...                 ...            ...         ...  
476381              NaN           7.04        0.00  
476382              NaN           0.00        0.00  
476383              NaN           3.90        0.00  
476384              NaN           3.08        0.00  
476385              NaN           6.84        0.00  

[476386 rows x 7 columns]

In [18]:
# Iterate over the records in the dataframe
t0 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
    # print(f"Sent: {message}")
t1 = time.time()
print(f'Send took {(t1 - t0):.2f} seconds')

producer.flush()

t2 = time.time()
print(f'Flush took {(t2 - t1):.2f} seconds')


Send took 60.16 seconds
Flush took 0.01 seconds


In [19]:
print(f'total time: {(t2 - t0):.2f} seconds')


total time: 60.16 seconds
